# Model Training

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# Modelling
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import warnings

In [2]:
df = pd.read_csv(f"data/raw.csv")

In [3]:
df.head(3)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93


In [4]:
X = df.drop(columns="math_score")
y = df['math_score']

In [5]:
# Create column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)

In [6]:
X = preprocessor.fit_transform(X)

In [7]:
# Seperate dataset into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
x_train.shape, x_test.shape

((800, 19), (200, 19))

## Creating Evaluation Function to give all metrics after Model Training

In [8]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [9]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbours Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)

    # Making Predictions
    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)

    # Evaluating training and testing dataset
    trained_mae, trained_rmse, trained_r2 = evaluate_model(y_train,y_train_pred)
    test_mae, test_rmse, test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Performance for training set")
    print(f"Mean Absolute Error : {trained_mae}")
    print(f"Root Mean Squared Error : {trained_rmse}")
    print(f"R2 Score : {trained_r2}")

    print('---------------------------------------------')

    print("Model Performance for testing set")
    print(f"Mean Absolute Error : {test_mae}")
    print(f"Root Mean Squared Error : {test_rmse}")
    print(f"R2 Score : {test_r2}")
    r2_list.append(test_r2)

    print('='*35)
    print()

Linear Regression
Model Performance for training set
Mean Absolute Error : 4.272265625
Root Mean Squared Error : 5.340183842955456
R2 Score : 0.8735068463594741
---------------------------------------------
Model Performance for testing set
Mean Absolute Error : 4.2253125
Root Mean Squared Error : 5.421371554436202
R2 Score : 0.8792164757750934

Lasso
Model Performance for training set
Mean Absolute Error : 5.206302661246526
Root Mean Squared Error : 6.593815587795566
R2 Score : 0.8071462015863456
---------------------------------------------
Model Performance for testing set
Mean Absolute Error : 5.157881810347763
Root Mean Squared Error : 6.519694535667419
R2 Score : 0.8253197323627853

Ridge
Model Performance for training set
Mean Absolute Error : 4.26498782372598
Root Mean Squared Error : 5.323324922741654
R2 Score : 0.8743042615212909
---------------------------------------------
Model Performance for testing set
Mean Absolute Error : 4.211100688014259
Root Mean Squared Error : 5.

## Results

In [11]:
pd.DataFrame(list(zip(model_list,r2_list)), columns=["Model Name","R2 Score"]).sort_values(by=["R2 Score"],ascending=False)

,Model Name,R2 Score
2,Ridge,0.880593
0,Linear Regression,0.879216
5,Random Forest Regressor,0.852895
8,AdaBoost Regressor,0.851692
7,CatBoosting Regressor,0.851632
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-Neighbours Regressor,0.783497
4,Decision Tree,0.705513
